<a href="https://colab.research.google.com/github/albey-code/hippoabstraction/blob/main/events_duration1s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook addresses the fact that in Mona Garvert's `mydesign_1007.m` and `mydesign_1012.m` a **blocklength of 1s was specified as an event duration for all objects and the button press.**

Previously, when converting the event files into .tsv, I assumed all events to be instantaneous, so 0s. Here, I will change the duration for all events to 1s for further use in the GLM.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
from pathlib import Path

In [13]:
base_path = '/content/drive/MyDrive'

old_events_path = Path(base_path, 'events_old')  # All events incl. object and button press with a duration of 0 seconds
new_events_path = Path(base_path, 'events')      # All events incl. object and button press with a duration of 1 second

In [14]:
subjects = range(1, 24)   # range(1, 24) if files start at sub01
runs = range(1, 4)

# For patch and no-patch trials together

In [15]:
# Check whether all old event files have duration of 0 seconds

summary = []

for sid in subjects:
    for run in runs:
        src = old_events_path / f"sub{sid:02d}_run{run}_events.tsv"
        if not src.exists():
            continue

        df = pd.read_csv(src, sep='\t')

        if 'duration' in df.columns:
            n_total = len(df)
            n_zero = (df['duration'] == 0).sum()
            summary.append((src.name, n_total, n_zero))
        else:
            summary.append((src.name, None, "no duration column"))

# Display summary
for fname, total, zero in summary:
    print(f"{fname:30s} | total={total} | duration=0 → {zero}")

sub01_run1_events.tsv          | total=351 | duration=0 → 351
sub01_run2_events.tsv          | total=346 | duration=0 → 346
sub01_run3_events.tsv          | total=351 | duration=0 → 351
sub02_run1_events.tsv          | total=346 | duration=0 → 346
sub02_run2_events.tsv          | total=349 | duration=0 → 349
sub02_run3_events.tsv          | total=347 | duration=0 → 347
sub03_run1_events.tsv          | total=348 | duration=0 → 348
sub03_run2_events.tsv          | total=345 | duration=0 → 345
sub03_run3_events.tsv          | total=348 | duration=0 → 348
sub04_run1_events.tsv          | total=351 | duration=0 → 351
sub04_run2_events.tsv          | total=346 | duration=0 → 346
sub04_run3_events.tsv          | total=356 | duration=0 → 356
sub05_run1_events.tsv          | total=349 | duration=0 → 349
sub05_run2_events.tsv          | total=348 | duration=0 → 348
sub05_run3_events.tsv          | total=351 | duration=0 → 351
sub06_run1_events.tsv          | total=348 | duration=0 → 348
sub06_ru

In [19]:
# Force new duration to be 1 second for all event files

processed, missing, no_folder = [], [], []

for sid in subjects:
    for run in runs:
        src = old_events_path / f"sub{sid:02d}_run{run}_events.tsv"
        if not src.exists():
            missing.append(src.as_posix())
            continue

        df = pd.read_csv(src, sep='\t')
        df['duration'] = 1.0  # overwrite all durations

        out_dir = new_events_path / f"subject_{sid:02d}"
        if not out_dir.exists():
            no_folder.append(out_dir.as_posix())
            continue  # skip saving if folder doesn’t exist

        dst = out_dir / src.name
        df.to_csv(dst, sep='\t', index=False)
        processed.append(dst.as_posix())

print(f"Wrote {len(processed)} updated event files.")
if missing:
    print(f"Missing {len(missing)} source files, e.g. {missing[:3]}")
if no_folder:
    print(f"Target folder not found for {len(no_folder)} subjects, e.g. {no_folder[:3]}")

Wrote 69 updated event files.


Check that all durations are 1 second across all 23 subjects and all 3 runs.

In [20]:
# Check the duration is 1 second for all new events.tsv files

all_good, issues = [], []

for sid in subjects:
    for run in runs:
        fname = new_events_path / f"subject_{sid:02d}" / f"sub{sid:02d}_run{run}_events.tsv"
        if not fname.exists():
            issues.append((fname.as_posix(), "file missing"))
            continue

        df = pd.read_csv(fname, sep='\t')
        if (df['duration'] == 1.0).all():
            all_good.append(fname.as_posix())
        else:
            issues.append((fname.as_posix(), "not all durations == 1"))

print(f"{len(all_good)} files OK (all durations = 1.0).")
if issues:
    print(f"{len(issues)} files with problems:")
    for fn, msg in issues[:5]:  # only show first 5 issues
        print("  ", fn, "→", msg)

69 files OK (all durations = 1.0).


In [21]:
fname = new_events_path / "subject_01" / "sub01_run1_events.tsv"

df = pd.read_csv(fname, sep='\t')
print("Columns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

Columns: ['onset', 'duration', 'trial_type']

First few rows:
    onset  duration    trial_type
0   3.061       1.0  button_press
1   3.570       1.0     object_10
2   4.845       1.0  button_press
3   7.131       1.0  button_press
4  11.690       1.0      object_6


In [23]:
fname = old_events_path / "sub01_run1_events.tsv"

df = pd.read_csv(fname, sep='\t')
print("Columns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

Columns: ['onset', 'duration', 'trial_type']

First few rows:
    onset  duration    trial_type
0   3.061       0.0  button_press
1   3.570       0.0     object_10
2   4.845       0.0  button_press
3   7.131       0.0  button_press
4  11.690       0.0      object_6


# For only no-patch trials

In [24]:
# Check whether all old event files have duration of 0 seconds

summary = []

for sid in subjects:
    for run in runs:
        src = old_events_path / f"sub{sid:02d}_run{run}_events_np.tsv" # Add the _np ending to indicate no_patch trials!!
        if not src.exists():
            continue

        df = pd.read_csv(src, sep='\t')

        if 'duration' in df.columns:
            n_total = len(df)
            n_zero = (df['duration'] == 0).sum()
            summary.append((src.name, n_total, n_zero))
        else:
            summary.append((src.name, None, "no duration column"))

# Display summary
for fname, total, zero in summary:
    print(f"{fname:30s} | total={total} | duration=0 → {zero}")

sub01_run1_events_np.tsv       | total=191 | duration=0 → 191
sub01_run2_events_np.tsv       | total=195 | duration=0 → 195
sub01_run3_events_np.tsv       | total=203 | duration=0 → 203
sub02_run1_events_np.tsv       | total=191 | duration=0 → 191
sub02_run2_events_np.tsv       | total=197 | duration=0 → 197
sub02_run3_events_np.tsv       | total=202 | duration=0 → 202
sub03_run1_events_np.tsv       | total=196 | duration=0 → 196
sub03_run2_events_np.tsv       | total=191 | duration=0 → 191
sub03_run3_events_np.tsv       | total=193 | duration=0 → 193
sub04_run1_events_np.tsv       | total=192 | duration=0 → 192
sub04_run2_events_np.tsv       | total=201 | duration=0 → 201
sub04_run3_events_np.tsv       | total=197 | duration=0 → 197
sub05_run1_events_np.tsv       | total=198 | duration=0 → 198
sub05_run2_events_np.tsv       | total=190 | duration=0 → 190
sub05_run3_events_np.tsv       | total=189 | duration=0 → 189
sub06_run1_events_np.tsv       | total=197 | duration=0 → 197
sub06_ru

In [25]:
# Force new duration to be 1 second for all no_patch event files

processed, missing, no_folder = [], [], []

for sid in subjects:
    for run in runs:
        # look for the _np files only
        src = old_events_path / f"sub{sid:02d}_run{run}_events_np.tsv"  # Only this line was changed to _np
        if not src.exists():
            missing.append(src.as_posix())
            continue

        df = pd.read_csv(src, sep='\t')
        df['duration'] = 1.0  # overwrite all durations

        out_dir = new_events_path / f"subject_{sid:02d}"
        if not out_dir.exists():
            no_folder.append(out_dir.as_posix())
            continue  # skip saving if folder doesn’t exist

        dst = out_dir / src.name
        df.to_csv(dst, sep='\t', index=False)
        processed.append(dst.as_posix())

print(f"Wrote {len(processed)} updated _np event files.")
if missing:
    print(f"Missing {len(missing)} source files, e.g. {missing[:3]}")
if no_folder:
    print(f"Target folder not found for {len(no_folder)} subjects, e.g. {no_folder[:3]}")

Wrote 69 updated _np event files.


Check that all durations are 1 second across all 23 subjects and all 3 runs for no-patch (_np) trials.

In [26]:
# Check the duration is 1 second for all new events.tsv files

all_good, issues = [], []

for sid in subjects:
    for run in runs:
        fname = new_events_path / f"subject_{sid:02d}" / f"sub{sid:02d}_run{run}_events_np.tsv"
        if not fname.exists():
            issues.append((fname.as_posix(), "file missing"))
            continue

        df = pd.read_csv(fname, sep='\t')
        if (df['duration'] == 1.0).all():
            all_good.append(fname.as_posix())
        else:
            issues.append((fname.as_posix(), "not all durations == 1"))

print(f"{len(all_good)} files OK (all durations = 1.0).")
if issues:
    print(f"{len(issues)} files with problems:")
    for fn, msg in issues[:5]:  # only show first 5 issues
        print("  ", fn, "→", msg)

69 files OK (all durations = 1.0).


In [27]:
fname = new_events_path / "subject_01" / "sub01_run1_events_np.tsv"

df = pd.read_csv(fname, sep='\t')
print("Columns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

Columns: ['onset', 'duration', 'trial_type']

First few rows:
    onset  duration    trial_type
0   3.061       1.0  button_press
1   4.845       1.0  button_press
2   7.131       1.0  button_press
3  11.690       1.0      object_6
4  16.730       1.0      object_4


In [28]:
fname = old_events_path / "sub01_run1_events_np.tsv"

df = pd.read_csv(fname, sep='\t')
print("Columns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

Columns: ['onset', 'duration', 'trial_type']

First few rows:
    onset  duration    trial_type
0   3.061       0.0  button_press
1   4.845       0.0  button_press
2   7.131       0.0  button_press
3  11.690       0.0      object_6
4  16.730       0.0      object_4
